P(y|X) = [P(x1|y) P(x2|y) ... P(y)]/P(X)  X is the feature vector X= (x1,x2,x3.....,xn) all x are IID , using the chain rule

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets

import pandas as pd                                         #Data scientist starter pack
from matplotlib import pyplot as plt
import math

from collections import Counter                             #Some useful tools 
from functools import reduce

import seaborn as sns                                       #More on plotting

import torch as trc                                         #To set PyTorch framework
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv1d, MaxPool1d, Module, Softmax, BatchNorm1d, Dropout
from torch.optim import Adam, SGD

from sklearn.model_selection import train_test_split        #Tool to fast build validation set

from datetime import datetime                               #To deal with datetime objects

import string                                               #Some useful string tools
import nltk                                                 #Natural language processing libraries
import spacy
import re                                                   #Regex tools

import random as rdm                                        #Simulation tools
import time                                                 #Ticking away the moments that make up a dull day

from tqdm import tqdm                                       #Te quiero desmasiado 

In [ ]:
dow_jones = pd.read_csv('datasets/upload_DJIA_table.csv', usecols=['Date','Open','Close'])

In [ ]:
ground_truth = []
for idx in dow_jones.index:
    if dow_jones.iloc[idx].Close >= dow_jones.iloc[idx].Open:
        ground_truth.append(1)
    else:
        ground_truth.append(0)

In [ ]:
dow_jones['Label'] = ground_truth

In [ ]:
news = pd.read_csv('datasets/RedditNews.csv')

In [ ]:
def text_preprocesser(text:string)->list:
    tokenized = nltk.word_tokenize(''.join([char 
                                       for char 
                                       in text.lower() 
                                       if char not in string.punctuation]))
                                       
    stop_words = nltk.corpus.stopwords.words('english')
    no_stop_words = [word 
                     for word 
                     in tokenized 
                     if word not in stop_words]

    stemmer = nltk.stem.porter.PorterStemmer()

    return [stemmer.stem(word)
            for word 
            in no_stop_words]

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
documents = {}
for date in tqdm(news.Date.unique()):
    processed = []
    for text in news.News[news.Date == date]:
        processed += text_preprocesser(text)
    documents[date] = processed

In [ ]:
final_dataset = dow_jones.merge(newers)
final_dataset = final_dataset.drop(labels=['Open','Close'], axis=1)

Naive Bayes classification based on Bayes Theorem
P(y|X) = ( P(X|y) P(y) ) / P(X)
X is the vector of features and all are mutually indipendent
P(y|X) = ( P(x1|y) P(x2|y) P(x3|y) ... P(xn|y) P(y) ) / P(X)
P(y|X) is the posterior probability
P(y) is the prior probability
The goal is to select the class with highest probability
y = max(P(y|X)) 
since P(x1|y) P(x2|y) ... are bounded 0 to 1  and if are multiplied , will get very small number. so its suggest to use log
log(P(x1|y)) + log(P(x2|y)) ....

To calculate the conditional prob P(xn|y) is used the Bernoulli distribution
P(X) = p if X =1 and q if X = 0
where q = 1-p and 0 < p <1

In [ ]:
def accuracy(y_t,y_p):
    return np.sum(y_t == y_p) / len(y_t)

In [60]:
class BernoulliNaiveBayes:
    def fit(self, X, y):
        self.classes = np.unique(y) # trova le classi uniche
        self.parameters = []
        for c in self.classes:
            X_c = X[y == c]  # seleziona le osservazioni di classe c
            parameters_c = {}
            for i in range(X.shape[1]):
                parameters_c[i] = (X_c[:, i].sum() + 1) / (X_c.shape[0] + 2) # calcola la probabilità di ogni caratteristica per la classe c
            self.parameters.append(parameters_c)
        
    def predict_proba(self, X):
        probas = []
        for i, x in enumerate(X):
            proba_c = []
            for c, p in zip(self.classes, self.parameters):
                proba = np.prod([p[i]**x * (1 - p[i])**(1 - x) for i, x in enumerate(x)]) 
                # calcola la probabilità della classe c per l'osservazione x
                proba_c.append(proba)
            probas.append(proba_c)
        return probas
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

In [76]:
class BernoulliNBChat:
       
    def fit(self, X, y):
        self.classes_ = np.unique(y) # Get the unique class labels
        self.class_counts_ = np.zeros(len(self.classes_)) # Initialize a count for each class
        self.feature_counts_ = np.zeros((len(self.classes_), X.shape[1])) # Initialize a count for each feature given each class
        
        # Count the number of occurrences of each class and each feature given each class
        for i, c in enumerate(self.classes_):
            X_c = X[y == c] # Get the training samples for class c
            self.class_counts_[i] = X_c.shape[0] # Count the number of samples in class c
            self.feature_counts_[i, :] = (X_c == 1).sum(axis=0) # Count the number of 1s for each feature given class c
          
        # Calculate the probability of each class
        self.class_priors_ = self.class_counts_ / self.class_counts_.sum()
        
        # Calculate the probability of each feature given each class
        self.feature_probs_ = self.feature_counts_ / self.class_counts_[:, np.newaxis]
    
    def predict(self, X):
            # Initialize a matrix to store the log probability of each sample given each class
        log_probs = np.zeros((X.shape[0], len(self.classes_)))
        
        # Calculate the log probability of each sample given each class
        for i, c in enumerate(self.classes_):
            # Calculate the log probability of each feature given class c
            feature_log_probs = np.log(self.feature_probs_[i, :]) * X + np.log(1 - self.feature_probs_[i, :]) * (1 - X)
            
            # Sum the log probabilities of all features to get the log probability of the sample given class c
            log_probs[:, i] = np.sum(feature_log_probs, axis=1)
        
        # Add the log probability of each class to the log probability of the sample given the class
        log_probs += np.log(self.class_priors_)
        
        # Predict the class with the highest log probability for each sample
        return self.classes_[np.argmax(log_probs, axis=1)]



In [86]:

from sklearn.naive_bayes import BernoulliNB

X,y =datasets.make_classification(n_samples=10000,n_features=10,n_classes=2)
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

classifier = BernoulliNaiveBayes()
classifier.fit(X_train,y_train)
preds = classifier.predict(X_test)

print("Accuracy ",accuracy(y_test,preds))

bnb = BernoulliNB()
bnb.fit(X_train,y_train)
predictions = bnb.predict(X_test)
print("Accuracy BNB",accuracy(y_test,predictions))

bernoulliNBChat = BernoulliNBChat()
bernoulliNBChat.fit(X_train,y_train)
predictions = bernoulliNBChat.predict(X_test)
print("Accuracy bernoulliNBChat",accuracy(y_test,predictions))



C:\Users\andrea.messina\AppData\Local\Temp\ipykernel_6120\821885837.py:19: RuntimeWarning: invalid value encountered in double_scalars
  proba = np.prod([p[i]**x * (1 - p[i])**(1 - x) for i, x in enumerate(x)])


Accuracy  0.5165
Accuracy BNB 0.9235
Accuracy bernoulliNBChat 0.5165


C:\Users\andrea.messina\AppData\Local\Temp\ipykernel_6120\3291593071.py:27: RuntimeWarning: divide by zero encountered in log
  feature_log_probs = np.log(self.feature_probs_[i, :]) * X + np.log(1 - self.feature_probs_[i, :]) * (1 - X)
C:\Users\andrea.messina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
